# Classical Equivalent Layer Magnetic data with positivity constraint

This notebook performs a classical equivalent source technique for a set of bodies using the positivity constraint.

#### Importing libraries

In [ ]:
% matplotlib inline 

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import datetime

from fatiando.gridder import regular
from fatiando.utils import ang2vec, vec2ang
from fatiando.mesher import Sphere, PointGrid,Prism
from fatiando.gravmag import sphere,prism
from fatiando.constants import CM, T2NT, G, SI2MGAL

In [ ]:
notebook_name = 'EQL_mag_positivity_constraint.ipynb'

### Black background figures

In [ ]:
plt.style.use('classic')

#### Importing auxiliary functions

In [ ]:
dir_modules = '../'
sys.path.append(dir_modules)

In [ ]:
import eql_functions as func

In [ ]:
import auxiliary_functions as fc

#### Loading properties of the model

In [ ]:
with open('../data/model_sphere_remanence.pickle') as f:
        model_sphere_remanence = pickle.load(f)

In [ ]:
with open('../data/model_sphere_induced.pickle') as f:
        model_sphere_induced = pickle.load(f)

#### Loading properties grid

In [ ]:
with open('../data/regular_grid.pickle') as f:
        regular_grid = pickle.load(f)

In [ ]:
with open('../data/airborne_survey.pickle') as f:
        airborne = pickle.load(f)

#### Loading data

In [ ]:
with open('../data/data_set.pickle') as f:
        data = pickle.load(f)

### Saving files

In [ ]:
saved_files = []

## Observation area

In [ ]:
print 'Area limits: \n x_max = %.1f m \n x_min = %.1f m \n y_max = %.1f m \n y_min = %.1f m' % (regular_grid['area'][1], 
                                                                                                regular_grid['area'][0],
                                                                                                regular_grid['area'][3],
                                                                                                regular_grid['area'][2])

### Regular grid information

In [ ]:
print 'Shape : (%.0f,%.0f)'% regular_grid['shape'] 
print 'Number of data: %.1f' % regular_grid['N']
print 'dx: %.1f m' % regular_grid['dx']
print 'dy: %.1f m ' % regular_grid['dy']

### Airborne survey information

In [ ]:
print 'Shape : (%.0f,%.0f)'% airborne['shape'] 
print 'Number of data: %.1f' % airborne['N'] 
print 'dx: %.1f m' % airborne['dx']
print 'dy: %.1f m ' % airborne['dy']
print 'Height: %.1f m' % airborne['z_obs']

## Properties of the model

### Dimension and coordinate of the center

In [ ]:
print 'Properties Sphere:\n xc = %.1f m \n yc = %.1f m \n zc = %.1f m \n radius = %.1f m ' %  (model_sphere_remanence['xc'],model_sphere_remanence['yc'],model_sphere_remanence['zc'],model_sphere_remanence['radius'])

### Main field

In [ ]:
inc_gf,dec_gf = model_sphere_remanence['main_field']

In [ ]:
print'Main field inclination: %.1f degree' % inc_gf
print'Main field declination: %.1f degree' % dec_gf

### Magnetization direction

#### Remanent direction

In [ ]:
print 'Intensity: %.1f A/m' % model_sphere_remanence['m_R']
print 'Inclination: %.1f degree' % model_sphere_remanence['inc_R']
print 'Declination: %.1f degree' % model_sphere_remanence['dec_R']

In [ ]:
inc_R,dec_R = model_sphere_remanence['inc_R'],model_sphere_remanence['dec_R']

#### Induced direction

In [ ]:
print 'Intensity: %.1f A/m' % model_sphere_induced['m_I']
print 'Inclination: %.1f degree' % model_sphere_induced['inc_I']
print 'Declination: %.1f degree' % model_sphere_induced['dec_I']

## Generating the layer with my function 

In [ ]:
h = 120.

#### Generating a layer

In [ ]:
layer = func.PointGrid(regular_grid['area'],h,regular_grid['shape'])

In [ ]:
xs,ys,zs = regular(regular_grid['area'],regular_grid['shape'],h)

## Gauss-Newton Method for estimating a positive layer

In [ ]:
lamb = .1
itmax = 30
p0 = 1000.*np.ones(regular_grid['Nx']*regular_grid['Ny'])
inc0,dec0 = inc_R,dec_R

In [ ]:
m_pos,inc_pos,dec_pos,_,_,_,_,_ = fc.gauss_newton_positivity(data['tfa_obs_sphere_remanence_reg'],
                                                            regular_grid['x'],regular_grid['y'],regular_grid['z'],
                                                            xs,ys,zs,inc_gf,dec_gf,
                                                            p0,inc0,dec0,itmax,lamb)

In [ ]:
print inc_pos

In [ ]:
print dec_pos

In [ ]:
print m_pos

In [ ]:
print np.min(m_pos)

In [ ]:
tf_pred_pos = fc.tfa_layer(regular_grid['x'],regular_grid['y'],regular_grid['z'],
                           xs,ys,zs,inc_gf,dec_gf,m_pos,inc_pos,dec_pos)

### Calculating residual

In [ ]:
r_pos = data['tfa_obs_sphere_remanence_reg'] - tf_pred_pos

#### Parameters for normalized histogram

In [ ]:
r_norm,r_mean,r_std = func.residual(data['tfa_obs_sphere_remanence_reg'],tf_pred_pos)

## Comparison between the Model and the Layer with posivity

In [ ]:
title_font = 20
bottom_font = 16
saturation_factor = 1.
plt.close('all')
plt.figure(figsize=(19,7), tight_layout=True)

ranges = saturation_factor*np.abs([data['tfa_obs_sphere_remanence_reg'].max(), data['tfa_obs_sphere_remanence_reg'].min(),
                                   tf_pred_pos.max(), tf_pred_pos.min()]).max()
plt.subplot(1,4,1)
plt.contourf(regular_grid['y'].reshape(regular_grid['shape']),
             regular_grid['x'].reshape(regular_grid['shape']), 
             data['tfa_obs_sphere_remanence_reg'].reshape(regular_grid['shape']),
             20, cmap='Greys',vmin=-ranges, vmax=ranges)
plt.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('nT')
plt.xlabel('y (m)', fontsize = title_font)
plt.ylabel('x (m)', fontsize = title_font)
plt.title('TFA (observed) ', fontsize=title_font)

plt.subplot(1,4,2)
plt.contourf(regular_grid['y'].reshape(regular_grid['shape']),
             regular_grid['x'].reshape(regular_grid['shape']),
             tf_pred_pos.reshape(regular_grid['shape']),
             20, cmap='Greys',  vmin=-ranges, vmax=ranges)
plt.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('nT')
plt.xlabel('y (m)', fontsize = title_font)
plt.ylabel('x (m)', fontsize = title_font)
plt.title('TFA (Layer)', fontsize=title_font)

plt.subplot(1,4,3)
plt.contourf(regular_grid['y'].reshape(regular_grid['shape']),
             regular_grid['x'].reshape(regular_grid['shape']),
             r_pos.reshape(regular_grid['shape']), 
             20, cmap='Greys',  vmin=-ranges, vmax=ranges)
plt.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('nT')
plt.xlabel('y (m)', fontsize = title_font)
plt.ylabel('x (m)', fontsize = title_font)
plt.title('Residual map', fontsize=title_font)


plt.subplot(1,4,4)
plt.xlabel('$\mu$ = %.3f | $\sigma$ = %.3f nT' % (r_mean, r_std),
           fontsize = bottom_font)
nbins = int((np.max(r_norm - np.min(r_norm))))
plt.hist(r_norm,bins=nbins,normed=True)
plt.ylim(0.,1.)
plt.xlim(-10., 10.)
plt.title('Normalized Histogram',fontsize=title_font)

#file_name = 'figs/eqlayer_with_positivity_sphere'
#plt.savefig(file_name+'.png',dpi=200)
#saved_files.append(file_name+'.png')


plt.show()

### Magnetic moment within the Layer with positivity constraint

In [ ]:
title_font = 20
bottom_font = 16
saturation_factor = 1.
plt.close('all')
plt.figure(figsize=(8,8), tight_layout=True)

ranges = saturation_factor*np.abs([m_pos.max(), m_pos.min()]).max()

plt.contourf(regular_grid['y'].reshape(regular_grid['shape']),
             regular_grid['x'].reshape(regular_grid['shape']), 
             m_pos.reshape(regular_grid['shape']),
             20, cmap='Greys_r',vmin=-ranges, vmax=ranges)
plt.colorbar(pad=0.01, aspect=40, shrink=1.0).set_label('$A.m^2$')
plt.xlabel('y (m)', fontsize = title_font)
plt.ylabel('x (m)', fontsize = title_font)
plt.annotate('Magnetic Moment (Layer)', xy = (0.05, 0.93),xycoords = 'axes fraction', fontsize=10)

#file_name = 'figs/magnetic_moment_positive'
#plt.savefig(file_name+'.png',dpi=200)
#saved_files.append(file_name+'.png')

plt.show